In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, \
    mean_squared_log_error, r2_score

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool

import pickle

import warnings
warnings.filterwarnings('ignore')

RAND = 10

In [2]:
df_clean = pd.read_csv(r'C:\Users\main6\OneDrive\Документы\jupyter\Pet_pro\data\df_clean.csv')

In [3]:
df_clean[:5]

,ID,LAT,LON,Precipitation,LST,AAI,CloudFraction,NO2_strat,TropopausePressure,month,year,NO2_ratio,Sum_Concentration,target
0,PD01,45.601585,11.903551,0.000000,NaN,0.230527,0.559117,0.000024,14440.82126,1,2019,NaN,NaN,31.0
1,PD04,45.371005,11.840830,3.047342,NaN,-0.074006,0.869309,0.000024,14441.79815,1,2019,NaN,NaN,42.0
2,RO01,45.045825,12.060869,0.000000,NaN,0.024470,0.674160,0.000024,14437.38294,1,2019,NaN,NaN,31.0
3,RO02,45.104075,11.553241,1.200467,NaN,-0.010442,0.920054,0.000024,14440.83831,1,2019,NaN,NaN,30.0
4,RO03,45.038758,11.790152,1.274564,NaN,-0.176178,0.747464,0.000024,14438.79037,1,2019,NaN,NaN,58.0


In [4]:
# прологорифмируем целевую переменную, т.к. распределение не нормальное
df_clean['target_log'] = np.log(df_clean.target + 1)

# удалим столбец target
df_clean = df_clean.drop('target', axis=1)

In [5]:
# создадим датасет с категориальными данными
df_cat = df_clean.copy()

cat_columns = df_cat.select_dtypes('object').columns
df_cat[cat_columns] = df_cat[cat_columns].astype('category')

In [6]:
# бинаризуем данные
df_label = pd.get_dummies(df_clean, drop_first=True)

In [7]:
def r2_adjusted(y_true: np.ndarray, y_pred: np.ndarray,
                X_test: np.ndarray) -> float:
    """Коэффициент детерминации (множественная регрессия)"""
    N_objects = len(y_true)
    N_features = X_test.shape[1]
    r2 = r2_score(y_true, y_pred)
    return 1 - (1 - r2) * (N_objects - 1) / (N_objects - N_features - 1)


def mpe(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean percentage error"""
    return np.mean((y_true - y_pred) / y_true) * 100


def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean absolute percentage error"""
    return np.mean(np.abs((y_pred - y_true) / y_true)) * 100


def wape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Weighted Absolute Percent Error"""
    return np.sum(np.abs(y_pred - y_true)) / np.sum(y_true) * 100


def rmsle(y_true: np.ndarray, y_pred: np.ndarray) -> np.float64:
    """
    The Root Mean Squared Log Error (RMSLE) metric 
    Логарифмическая ошибка средней квадратичной ошибки
    """
    try:
        return np.sqrt(mean_squared_log_error(y_true, y_pred))
    except:
        return None


def get_metrics_regression(y_test: np.ndarray,
                           y_pred: np.ndarray,
                           X_test: np.ndarray,
                           name: str = None):
    """Генерация таблицы с метриками для задачи регрессии"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]

    df_metrics['MAE'] = mean_absolute_error(y_test, y_pred)
    df_metrics['MSE'] = mean_squared_error(y_test, y_pred)
    df_metrics['RMSE'] = np.sqrt(mean_squared_error(y_test, y_pred))
    df_metrics['RMSLE'] = rmsle(y_test, y_pred)
    df_metrics['R2 adjusted'] = r2_adjusted(y_test, y_pred, X_test)
    df_metrics['MPE_%'] = mpe(y_test, y_pred)
    df_metrics['MAPE_%'] = mape(y_test, y_pred)
    df_metrics['WAPE_%'] = wape(y_test, y_pred)

    return df_metrics

In [8]:
def check_overfitting(model, X_train, y_train, X_test, y_test):
    """
    Проверка на overfitting для регрессии при помощи
    метрики Mean absolute error
    """
    y_pred_train = np.exp(model.predict(X_train)) - 1
    y_pred_test = np.exp(model.predict(X_test)) - 1
    value_train = mean_absolute_error(y_train, y_pred_train)
    value_test = mean_absolute_error(y_test, y_pred_test)

    print(f'Mean absolute error train: %.3f' % value_train)
    print(f'Mean absolute error test: %.3f' % value_test)
    print(f'delta = {(abs(value_train - value_test)/value_test*100):.1f} %')

In [9]:
def fillna_X(X_train, X_test, list_median, list_mean):
    """
    Функция заполнения пропусков разными значениями
    ----------------
    X_train, X_test: признаки после train_test_split
    list_median: список с признаками, необходимых заполнить медианой
    list_mean: список с признаками, необходимых заполнить средними
    """
    for m in list_median:
        X_train[m] = X_train[m].fillna(X_train[m].median())
        X_test[m] = X_test[m].fillna(X_test[m].median())
        
    for n in list_mean:
        X_train[n] = X_train[n].fillna(X_train[n].mean())
        X_test[n] = X_test[n].fillna(X_test[n].mean())

In [10]:
# Признаки с выбросами заполним медианными значениями
list_median = ['CloudFraction', 'NO2_ratio', 'Sum_Concentration', 'TropopausePressure']

# Остальные числовые признаки заполним средними
list_mean = ['NO2_strat', 'LST', 'AAI']

In [11]:
# разобьем данные на переменные и train/val/test
X = df_label.drop('target_log', axis=1)
y = df_label['target_log']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=RAND)

X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                    y_train,
                                                    test_size=0.16,
                                                    shuffle=True,
                                                    random_state=RAND)

In [12]:
fillna_X(X_train, X_test, list_median, list_mean)

fillna_X(X_train_, X_val, list_median, list_mean)

y_train = y_train.fillna(y_train.median())
y_test = y_test.fillna(y_test.median())

y_train_ = y_train_.fillna(y_train_.median())
y_val = y_val.fillna(y_val.median())

# 1. Linear Regression

In [13]:
# стандартизируем данные
st = StandardScaler()
X_train_std = st.fit_transform(X_train)
X_test_std = st.transform(X_test)

# потенцирование целевой переменной 
y_test_exp = np.exp(y_test) - 1
y_train_exp = np.exp(y_train) - 1

In [14]:
lr = LinearRegression()
lr.fit(X_train_std, y_train)

LinearRegression()

In [15]:
y_pred = lr.predict(X_test_std)
y_pred_exp = np.exp(y_pred) - 1

metrics = get_metrics_regression(y_test_exp, y_pred_exp, X_test_std,
                                 name='LinearRegression_baseline')
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,LinearRegression_baseline,8.347072,156.85872,12.524325,0.455343,0.451267,-inf,inf,34.324438


In [16]:
# посмотрим, не переобучилась ли модель
check_overfitting(lr,
                  X_train_std,
                  y_train_exp,
                  X_test_std,
                  y_test_exp)

Mean absolute error train: 8.317
Mean absolute error test: 8.347
delta = 0.4 %


- так как у нашей целевой переменной  много выбросов, будем ориентироваться в первую очередь на MAE
- переобучение минимальное
- линейная регрессия отработала хорошо

# 2. Decision Tree Regressor

In [17]:
dtr = DecisionTreeRegressor(random_state=RAND)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(random_state=10)

In [18]:
y_pred = dtr.predict(X_test)
y_pred_exp = np.exp(y_pred) - 1

metrics = metrics.append(
    get_metrics_regression(y_test_exp, y_pred_exp, X_test,
                           name='DecisonTreeRegressor_baseline'))
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,LinearRegression_baseline,8.347072,156.858720,12.524325,0.455343,0.451267,-inf,inf,34.324438
0,DecisonTreeRegressor_baseline,10.133774,228.179589,15.105614,0.607437,0.201768,-inf,inf,41.671629


In [19]:
check_overfitting(dtr,
                  X_train,
                  y_train_exp,
                  X_test,
                  y_test_exp)

Mean absolute error train: 0.497
Mean absolute error test: 10.134
delta = 95.1 %


- показатель MAE ухудшился почти на 2 целые
- невероятно высокое переобучение
- дерево решений плохо справилось с данными

# 3. Random Forest Regressor

In [20]:
rf = RandomForestRegressor(random_state=RAND)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=10)

In [21]:
y_pred = rf.predict(X_test)
y_pred_exp = np.exp(y_pred) - 1


metrics = metrics.append(
    get_metrics_regression(y_test_exp, y_pred_exp, X_test,
                           name='RandomForestRegressor_baseline'))
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,LinearRegression_baseline,8.347072,156.858720,12.524325,0.455343,0.451267,-inf,inf,34.324438
0,DecisonTreeRegressor_baseline,10.133774,228.179589,15.105614,0.607437,0.201768,-inf,inf,41.671629
0,RandomForestRegressor_baseline,8.258138,176.019952,13.267251,0.476677,0.384236,-inf,inf,33.958726


In [22]:
check_overfitting(rf,
                  X_train,
                  y_train_exp,
                  X_test,
                  y_test_exp)

Mean absolute error train: 2.636
Mean absolute error test: 8.258
delta = 68.1 %


- метрика немного улучшилась по сравнению с линейной регрессией
- однако процент переобучения все также очень высок
- случайные леса выдали не очень хороший результат

# 4. XGBoost 

In [23]:
# потенцирование
y_train_exp_ = np.exp(y_train_) - 1

# создаем валидационный датасет
eval_set = [(X_val, y_val)]

In [24]:
xgb = XGBRegressor(random_state=RAND)

xgb.fit(X_train_,
        y_train_,
        eval_metric='mae',
        eval_set=eval_set,
        early_stopping_rounds=100,
        verbose=0)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=10, ...)

In [25]:
y_pred = xgb.predict(X_test)
y_pred_exp = np.exp(y_pred) - 1

metrics = metrics.append(
    get_metrics_regression(y_test_exp, y_pred_exp, X_test,
                           name='XGBoost_baseline'))
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,LinearRegression_baseline,8.347072,156.858720,12.524325,0.455343,0.451267,-inf,inf,34.324438
0,DecisonTreeRegressor_baseline,10.133774,228.179589,15.105614,0.607437,0.201768,-inf,inf,41.671629
0,RandomForestRegressor_baseline,8.258138,176.019952,13.267251,0.476677,0.384236,-inf,inf,33.958726
0,XGBoost_baseline,7.173608,121.679612,11.030848,0.398242,0.574333,-inf,inf,29.498972


In [26]:
check_overfitting(xgb,
                  X_train_,
                  y_train_exp_,
                  X_test,
                  y_test_exp)

Mean absolute error train: 5.710
Mean absolute error test: 7.174
delta = 20.4 %


- средняя абсолютная ошибка улучшилась на целую единицу
- переобучение ниже, чем в лесах и деревьях, но все равно еще высокое
- XGBoost справился неплохо, но все же хотелось бы увидеть более низкий overfitting

# 5. LightGBM

In [27]:
# разобьем так же датасет с категориальными данными на train/val/test
X = df_cat.drop(['target_log'], axis=1)
y = df_cat['target_log']

X_train_ct, X_test_ct, y_train_ct, y_test_ct = train_test_split(
    X, y, test_size=0.2, random_state=RAND)

X_ct, X_val_ct, y_ct, y_val_ct = train_test_split(
    X_train_ct, y_train_ct, test_size=0.16, random_state=RAND)

In [28]:
fillna_X(X_train_ct, X_test_ct, list_median, list_mean)
fillna_X(X_ct, X_val_ct, list_median, list_mean)

y_train_ct = y_train_ct.fillna(y_train_ct.median())
y_test_ct = y_test_ct.fillna(y_test_ct.median())

y_ct = y_ct.fillna(y_ct.median())
y_val_ct = y_val_ct.fillna(y_val_ct.median())

In [29]:
# потенцирование
y_train_ct_exp = np.exp(y_train_ct) - 1
y_test_ct_exp = np.exp(y_test_ct) - 1
y_ct_exp = np.exp(y_ct) - 1

# валидационный датасет
eval_ct = [(X_val_ct, y_val_ct)]

In [36]:
lgbm = LGBMRegressor(random_state=RAND)

lgbm.fit(X_ct,
         y_ct,
         eval_metric='mae',
         eval_set=eval_ct,
         verbose=0)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002367 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2297
[LightGBM] [Info] Number of data points in the train set: 58184, number of used features: 13
[LightGBM] [Info] Start training from score 3.028362


LGBMRegressor(random_state=10)

In [31]:
y_pred = lgbm.predict(X_test_ct)
y_pred_exp = np.exp(y_pred) - 1

metrics = metrics.append(
    get_metrics_regression(y_test_ct_exp, y_pred_exp, X_test_ct,
                           name='LightGBM_baseline'))
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,LinearRegression_baseline,8.347072,156.858720,12.524325,0.455343,0.451267,-inf,inf,34.324438
0,DecisonTreeRegressor_baseline,10.133774,228.179589,15.105614,0.607437,0.201768,-inf,inf,41.671629
0,RandomForestRegressor_baseline,8.258138,176.019952,13.267251,0.476677,0.384236,-inf,inf,33.958726
0,XGBoost_baseline,7.173608,121.679612,11.030848,0.398242,0.574333,-inf,inf,29.498972
0,LightGBM_baseline,7.120787,122.711309,11.077514,0.393043,0.573203,-inf,inf,29.304653


In [32]:
check_overfitting(lgbm,
                  X_ct,
                  y_ct_exp,
                  X_test_ct,
                  y_test_ct_exp)

Mean absolute error train: 6.203
Mean absolute error test: 7.121
delta = 12.9 %


- LightGBM отлично справляется с обучением 
- значение MAE еще ниже, а переобучение уменьшилось почти вдвое в сравнении с XGBoost

# 6. CatBoost

In [33]:
cat_features = X_val_ct.select_dtypes('category').columns.to_list()

In [34]:
cb = CatBoostRegressor(cat_features=cat_features,
                       random_state=RAND)

cb.fit(X_ct,
       y_ct,
       eval_set=eval_ct,
       verbose=False,
       early_stopping_rounds=100)

In [35]:
y_pred = cb.predict(X_test_ct)
y_pred_exp = np.exp(y_pred) - 1

metrics = metrics.append(
    get_metrics_regression(y_test_ct_exp, y_pred_exp, X_test_ct,
                           name='CatBoost_baseline'))
metrics

,model,MAE,MSE,RMSE,RMSLE,R2 adjusted,MPE_%,MAPE_%,WAPE_%
0,LinearRegression_baseline,8.347072,156.858720,12.524325,0.455343,0.451267,-inf,inf,34.324438
0,DecisonTreeRegressor_baseline,10.133774,228.179589,15.105614,0.607437,0.201768,-inf,inf,41.671629
0,RandomForestRegressor_baseline,8.258138,176.019952,13.267251,0.476677,0.384236,-inf,inf,33.958726
0,XGBoost_baseline,7.173608,121.679612,11.030848,0.398242,0.574333,-inf,inf,29.498972
0,LightGBM_baseline,7.120787,122.711309,11.077514,0.393043,0.573203,-inf,inf,29.304653
0,CatBoost_baseline,6.453979,99.260909,9.962977,0.359331,0.654765,-inf,inf,26.560497


In [36]:
check_overfitting(cb,
                  X_ct,
                  y_ct_exp,
                  X_test_ct,
                  y_test_ct_exp)

Mean absolute error train: 5.770
Mean absolute error test: 6.454
delta = 10.6 %


- MAE упала до рекордных шести целых
- переобучение еще ниже
- СatBoost выдал прекрасные результаты

In [38]:
# сохраним переменные для анализа важных признаков в конце
with open (r'C:\Users\main6\OneDrive\Документы\jupyter\Pet_pro\models\test_ct.pkl', 
           'wb') as f:
    pickle.dump((X_test_ct, y_test_ct), f)

with open(r'C:\Users\main6\OneDrive\Документы\jupyter\Pet_pro\models\models.pkl',
          'wb') as f:
    pickle.dump((lgbm, cb), f)

#### Общий вывод:

- линейная регрессия показала стандартные хорошие результаты
- результаты дерева решений и случайного леса очень слабые
- лучше всего отработал CatBoost
- в тюнинге будем использовать его, а так же LightGBM